In [1]:
import numpy as np
import pandas as pd
import math
import itertools
import matplotlib.pyplot as plt
import sys
import torch  
import numpy as np  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm 

import import_ipynb
from Environments import Square_Crossroads
from Actor_Critic import Actor_Critic

importing Jupyter notebook from Environments.ipynb
Environment reset with param
Environment reset with param
{0: [[0, 5], [5, 10], 0, 0], 1: [[5, 0], [0, 5], 0, 0], 2: [[5, 10], [5, 0], 0, 0], 3: [[10, 5], [5, 0], 0, 0]}
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
-599
{0: [[0.08860349964315856, 4.966599160633527], [5, 10], 0.09469000063047904, 339.34507444983115], 1: [[5, 0], [0, 5], 0.008788172970378573, 359.99998113091215], 2: [[5, 10], [5, 0], 0.04198821047919725, 7.242274145817169], 3: [[10, 5], [5, 0], 0.01798486468425529, 68.08403365717658]}
A car tried to go outside the boundaries
A car tried to go outside the boundaries
-399
{0: [[0.13694747027149834, 5.109351119429898], [5, 10], 0.15071582941520942, 71.29096235012538], 1: [[5.0659764663592926, 0.0019950259135468698], [0, 5], 0.06600662271056282, 1.7320076538509204], 2: [[5, 10], [5, 0], 0.09041518470155219, 0.7224464677227801], 3: [[10, 5], [5, 0],

RuntimeError: size mismatch, m1: [4 x 2], m2: [400 x 400] at C:\cb\pytorch_1000000000000\work\aten\src\TH/generic/THTensorMath.cpp:41

In [ ]:
n_times_steps = 3
seed = 10
dist_cars = 0.1

env = Square_Crossroads(n_times_steps, seed, dist_cars)
env.reset()
print(env.states)
action_space = {0:[0,0], 1:[0,0], 2:[0,0], 3:[0,0]}
# total_reward = 0

state_tensor = torch.zeros([4,2], dtype=torch.float32)


ac = Actor_Critic(state_tensor, 20, 400)

In [ ]:
def huber_loss(advantage, delta = 1.0):
    
    '''
    Huber loss, by give target and prediction
    
    '''
    
    error = abs(advantage)
    
    
    if error.item() > delta:
        loss = 0.5 * delta ** 2 + delta * (error - delta)

    else: 
        loss = 0.5 * error ** 2
#     loss = 0
    return loss

In [ ]:
hyperparameters = {"hidden_size_actor" : 200,
                   "hidden_size_critic" : 400,
                   "lr" : 4e-4,
                   "step_size" : 5,
                   "gamma" : 0.9}

max_episodes = 300
num_steps = 100
GAMMA = 0.99
delta = 0.1

In [ ]:
def train(env, hyperparameters):
    
    model = Actor_Critic(state_tensor, hyperparameters["hidden_size_actor"], hyperparameters["hidden_size_critic"])
    model_optimizer = optim.Adam(model.parameters(), lr = hyperparameters["lr"])
    model_scheduler = optim.lr_scheduler.StepLR(model_optimizer, step_size = hyperparameters["step_size"], gamma = hyperparameters["gamma"])
    
    all_lengths = []
    average_lengths = []
    all_rewards = []

    for episode in range(max_episodes):
        print("Training episode: ", episode)
        
        log_probs = []
        values = []
        rewards = []

        states = [v[0] for v in env.states.values()] 
        state_space = torch.tensor([[states[0][0], states[0][1]], 
                                    [states[1][0], states[1][1]],
                                    [states[2][0], states[2][1]],
                                    [states[3][0], states[3][1]]], dtype=torch.float32)

        for steps in tqdm(range(num_steps)):
#             print(f"Step {steps} of episode {episode}")
            
            value, policy_dist_alpha, policy_dist_beta = model.forward(state_space)
            
            value = torch.sum(value).detach().numpy()
            

            action_1 = np.random.beta(policy_dist_alpha[0].detach().numpy(), policy_dist_beta[0].detach().numpy(), size=2)
            action_2 = np.random.beta(policy_dist_alpha[1].detach().numpy(), policy_dist_beta[1].detach().numpy(), size=2)
            action_3 = np.random.beta(policy_dist_alpha[2].detach().numpy(), policy_dist_beta[2].detach().numpy(), size=2)
            action_4 = np.random.beta(policy_dist_alpha[3].detach().numpy(), policy_dist_beta[3].detach().numpy(), size=2)

            action_space = torch.tensor([action_1,
                                         action_2,
                                         action_3,
                                         action_4],
                                         dtype=torch.float32)

            new_state_space, reward, done = env.step(action_space)
            rewards.append(reward)
            values.append(value)

            
            states = [v[0] for v in env.states.values()] 
            new_state_space = torch.tensor([[states[0][0], states[0][1]], 
                                            [states[1][0], states[1][1]],
                                            [states[2][0], states[2][1]],
                                            [states[3][0], states[3][1]]], dtype=torch.float32)

            state_space = new_state_space
            
            if done or steps == num_steps-1:
                Qval, policy_dist_alpha, policy_dist_beta = model.forward(state_space)
                Qval = torch.sum(Qval).detach().numpy()
                all_rewards.append(np.sum(rewards))
                all_lengths.append(steps)
                average_lengths.append(np.mean(all_lengths[-10:]))
                if episode % 10 == 0:                    
                    sys.stdout.write("episode: {}, reward: {}, total length: {}, average length: {} \n".format(episode, np.sum(rewards), steps, average_lengths[-1]))
                print("Episode finished")
                break
        
        # compute Q values
        Qvals = np.zeros_like(values)

        for t in reversed(range(len(rewards))):
            Qval = rewards[t] + GAMMA * Qval
            Qvals[t] = Qval
        
        #update actor critic
        
        values = [torch.from_numpy(item).float() for item in values]
        values = torch.FloatTensor(values)

        Qvals = torch.FloatTensor(Qvals)


        advantage = (Qvals - values).mean()
        print(advantage)
        loss_critic = huber_loss(advantage, delta)
        
        loss_actor = (-(policy_dist_alpha.sum() + policy_dist_beta.sum() + 1e-5)) * advantage   
            
        total_loss = loss_actor + loss_critic
        
        print(total_loss)
        model_optimizer.zero_grad()
        total_loss.backward()
        model_optimizer.step()
        model_scheduler.step()
        
    

In [ ]:
train(env, hyperparameters)